# Knight's Tour Problem by “Dynamic” WARNSDORFF Algorithm

<p>Knight’s Tour (騎⼠旅程問題) 是指在按照西洋棋中騎士的規定走法走遍整個棋盤的每一個方格，而且每個網格只能夠經過一次。</p>
<p>如果使用 Depth-first Search algorithm (深度優先搜尋演算法)， 當路徑搜尋遇到 dead-end 的問題時，會採⽤ backtracking (回溯) ⽅式解決該問題。</p>
<p>但這種的計算效能不高，因此 Warnsdorff 提出一套演算法來提升 Knight’s Tour 搜尋的效能。<br>
雖然如此，問題是：“Static” Warnsdorff rules 仍然無法避免搜尋時可能遇到 dead-end，必須 “回溯” (backtracking) 搜尋。</p>
<p>因此，本篇內容使用 “Dynamic” Warnsdorff 演算法，藉由動態更新各棋盤格點的 degree值，<br>
來協助避開搜尋時遇到 dead-end 的問題，在無需回溯的情況下，快速求解騎⼠旅程問題。</p>

<p>在解決 Knight's Tour 問題之前，我們必須先來了解一下<span style="font-weight: bolder; margin: 0 10px;">Warnsdorff’s Rule:</span></p>
<ol>
    <li>我們可以開始於棋盤中的任一格</li>
    <li>我們一律移往附近且未曾前往過的格子 ( 因為每個棋格只能走一次 )</li>
</ol>

<p>再經由以下 <span style="font-weight: bolder; margin: 0 10px;">Algorithm</span> 找出騎士的路徑</p>

## Step 1: Creating a degree map 

<p>騎士的移動方式有 8 種，但受限於棋盤的邊界，每一格的移動方式就未必有 8 種那麼多。</p>
<p>Degree map 是將每一格可能的移動方式數目標註在每一格，例如位於四角的棋格只有 2 種移動方式，標註為 2。</p>

In [1]:
def create_degree_map(num):
    import numpy as np
    degree_map = np.ones((num,num))+7

    # 最上列 和 最下列 都先減 4(被邊界限制住: 上4 下 4)
    degree_map[[0,num-1]]-=4

    # 第一行 和 最後一行 都先減 4(被邊界限制住: 左4、右4)
    for i in range(1,num-1): degree_map[i][[0,num-1]]-=4
    degree_map[0][[0,num-1]]-=2; degree_map[num-1][[0,num-1]]-=2 #四角減2 因和上下重疊

    # 第二行及倒數第二行減 2 ( 左左上、左左下  及 右右上、右右下)
    for i in range(1,num-1): degree_map[i][[1,num-2]]-=2
    # 第二行及倒數第二行 的 第一列和最後一列減 1 (因之前處理過其中一個方向了)
    for i in [0,num-1]: degree_map[i][[1,num-2]]-=1

    # 第二列及倒數第二列減 2 ( 左左上、右右上  及 左左下 、右右下)
    for i in [1,num-2]: degree_map[i][list(range(1,num-1))]-=2
    # 第二列及倒數第二列 的 第一行及倒數第一行減 1 (因之前處理過其中一個方向了)
    for i in [1,num-2]: degree_map[i][[0,num-1]]-=1
    
    return degree_map

In [2]:
num = 8
degree_map = create_degree_map(num).copy()
print(degree_map)

[[2. 3. 4. 4. 4. 4. 3. 2.]
 [3. 4. 6. 6. 6. 6. 4. 3.]
 [4. 6. 8. 8. 8. 8. 6. 4.]
 [4. 6. 8. 8. 8. 8. 6. 4.]
 [4. 6. 8. 8. 8. 8. 6. 4.]
 [4. 6. 8. 8. 8. 8. 6. 4.]
 [3. 4. 6. 6. 6. 6. 4. 3.]
 [2. 3. 4. 4. 4. 4. 3. 2.]]


## STEP 2 : Creating the 8 possible moves.

<p>正如上一步所說，騎士的移動方式有 8 種，這邊我們要將這 8 種走法存成陣列方便我們做運算</p>

In [3]:
move = [(1,2),(2,1),(2,-1),(1,-2),(-1,-2),(-2,-1),(-2,1),(-1,2)]

# STEP 3 : Initiating the start position of Knight.

我們可以為騎士選擇任一棋格為起始點

In [4]:
import random
import numpy as np

start_site_col = random.randint(0,num-1)
start_site_row = random.randint(0,num-1)

In [5]:
print(f'Start site : ({start_site_col},{start_site_row})')

Start site : (7,2)


# STEP 4 : Looping for finding the Hamiltonian Path for Knight’s Tour.

<p>在這邊有幾項規則需要特別注意</p>
<ol>
    <li>確認每一格的移動是否會超出邊界</li>
    <li>找出騎士的下一個移動點</li>
    <li>在每一的移動必須更新我們的 degree map </li>
</ol>

### function

In [6]:
# 找出所有可走的移動方式
def available_move_way(start_site_col, start_site_row,degree_map):
    move = [(1,2),(2,1),(2,-1),(1,-2),(-1,-2),(-2,-1),(-2,1),(-1,2)]
    available=[]
    for i,j in move:
        if (start_site_col+i)<0 or (start_site_col+i)>=num or (start_site_row+j)<0 or (start_site_row+j)>=num:
            continue
        else:
            # 若可走的點的 degree 為 0，則將不列入可走範圍中
            if degree_map[start_site_col+i][start_site_row+j]==0:
                continue
            else:
                available.append((i,j))
    return available

In [7]:
available_move_way(start_site_col,start_site_row,degree_map)

[(-1, -2), (-2, -1), (-2, 1), (-1, 2)]

In [8]:
def print_out_function(array):
    for i in range(len(array)):
        for j in range(len(array[i])):
            print(array[i][j],end='\t')
        print()

### Code

In [9]:
Hamiltonian_Path = np.ones((num,num))

for i in range(num*num):
    # 先將每一步的次序紀錄在我們的路徑陣列中
    Hamiltonian_Path[start_site_col][start_site_row]+=i
    
    # 在 degree map 中，將每一次的起始點的 degree 設定為 0
    degree_map[start_site_col][start_site_row] = 0
    
    # 該起始點周圍的點也要同時做動態的更改，同時也要記錄它的位置是哪些點
    possible_move_way=[]
    for j,k in available_move_way(start_site_col,start_site_row,degree_map): 
        degree_map[start_site_col+j][start_site_row+k]-=1
        possible_move_way.append((start_site_col+j, start_site_row+k))
        
    # 選擇 degree 最小的棋格為移動方向； 若 degree 相同，選擇排序最接近的
    possible_move_degree = [degree_map[i][j] for i,j in possible_move_way]
    possible_move = {i:j for i,j in zip(possible_move_way,possible_move_degree)}
    possible_move_min = [i for i in possible_move if possible_move[i]==min(possible_move.values())]
    
    if len(possible_move_min)==0:
        break
    else:
        start_site_col,start_site_row = possible_move_min[0]
        
print_out_function(Hamiltonian_Path)

44.0	5.0	64.0	55.0	42.0	7.0	38.0	61.0	
53.0	26.0	43.0	6.0	59.0	62.0	41.0	8.0	
4.0	45.0	54.0	63.0	56.0	39.0	60.0	37.0	
25.0	52.0	27.0	58.0	35.0	50.0	9.0	40.0	
18.0	3.0	46.0	51.0	28.0	57.0	36.0	49.0	
21.0	24.0	19.0	34.0	47.0	32.0	13.0	10.0	
2.0	17.0	22.0	29.0	12.0	15.0	48.0	31.0	
23.0	20.0	1.0	16.0	33.0	30.0	11.0	14.0	


<h2 style="color:red">以上Code 的合併</h2>

In [10]:
import random
import numpy as np
import sys

#============================
#          Funciton
#============================
def create_degree_map(num):
    import numpy as np
    degree_map = np.ones((num,num))+7

    # 最上列 和 最下列 都先減 4(被邊界限制住: 上4 下 4)
    degree_map[[0,num-1]]-=4

    # 第一行 和 最後一行 都先減 4(被邊界限制住: 左4、右4)
    for i in range(1,num-1): degree_map[i][[0,num-1]]-=4
    degree_map[0][[0,num-1]]-=2; degree_map[num-1][[0,num-1]]-=2 #四角減2 因和上下重疊

    # 第二行及倒數第二行減 2 ( 左左上、左左下  及 右右上、右右下)
    for i in range(1,num-1): degree_map[i][[1,num-2]]-=2
    # 第二行及倒數第二行 的 第一列和最後一列減 1 (因之前處理過其中一個方向了)
    for i in [0,num-1]: degree_map[i][[1,num-2]]-=1

    # 第二列及倒數第二列減 2 ( 左左上、右右上  及 左左下 、右右下)
    for i in [1,num-2]: degree_map[i][list(range(1,num-1))]-=2
    # 第二列及倒數第二列 的 第一行及倒數第一行減 1 (因之前處理過其中一個方向了)
    for i in [1,num-2]: degree_map[i][[0,num-1]]-=1
    
    return degree_map

# 找出所有可走的移動方式
def available_move_way(start_site_col, start_site_row,degree_map):
    move = [(1,2),(2,1),(2,-1),(1,-2),(-1,-2),(-2,-1),(-2,1),(-1,2)]
    available=[]
    for i,j in move:
        if (start_site_col+i)<0 or (start_site_col+i)>=num or (start_site_row+j)<0 or (start_site_row+j)>=num:
            continue
        else:
            # 若可走的點的 degree 為 0，則將不列入可走範圍中
            if degree_map[start_site_col+i][start_site_row+j]==0:
                continue
            else:
                available.append((i,j))
    return available

def print_out_function(array):
    for i in range(len(array)):
        for j in range(len(array[i])):
            print(int(array[i][j]),end='\t')
        print()
#============================
#       Function End
#============================

num = int(input('Please input the size of chessboard : '))
if num<8 or num>30: 
    print('Your input is Out Of Range !!!')
    sys.exit()
    

start_site_col = random.randint(0,num-1)
start_site_row = random.randint(0,num-1)
print(f'Random Start site : \t({start_site_col},{start_site_row})')

degree_map = create_degree_map(num).copy()
print('\nDegree map : \n')
print_out_function(degree_map)

Hamiltonian_Path = np.ones((num,num))

for i in range(num*num):
    # 先將每一步的次序紀錄在我們的路徑陣列中
    Hamiltonian_Path[start_site_col][start_site_row]+=i
    
    # 在 degree map 中，將每一次的起始點的 degree 設定為 0
    degree_map[start_site_col][start_site_row] = 0
    
    # 該起始點周圍的點也要同時做動態的更改，同時也要記錄它的位置是哪些點
    possible_move_way=[]
    for j,k in available_move_way(start_site_col,start_site_row,degree_map): 
        degree_map[start_site_col+j][start_site_row+k]-=1
        possible_move_way.append((start_site_col+j, start_site_row+k))
        
    # 選擇 degree 最小的棋格為移動方向； 若 degree 相同，選擇排序最接近的
    possible_move_degree = [degree_map[i][j] for i,j in possible_move_way]
    possible_move = {i:j for i,j in zip(possible_move_way,possible_move_degree)}
    possible_move_min = [i for i in possible_move if possible_move[i]==min(possible_move.values())]
    
    if len(possible_move_min)==0: break
    else: start_site_col,start_site_row = possible_move_min[0]

print('\nHamiltonian_Path : \n')
print_out_function(Hamiltonian_Path)

Please input the size of chessboard : 8
Random Start site : 	(7,6)

Degree map : 

2	3	4	4	4	4	3	2	
3	4	6	6	6	6	4	3	
4	6	8	8	8	8	6	4	
4	6	8	8	8	8	6	4	
4	6	8	8	8	8	6	4	
4	6	8	8	8	8	6	4	
3	4	6	6	6	6	4	3	
2	3	4	4	4	4	3	2	

Hamiltonian_Path : 

64	31	12	35	60	49	10	37	
13	34	61	50	11	36	55	48	
30	63	32	59	54	51	38	9	
33	14	45	62	39	56	47	52	
44	29	58	25	46	53	8	23	
15	18	43	40	57	24	5	2	
28	41	20	17	26	3	22	7	
19	16	27	42	21	6	1	4	
